In [0]:
!pip install eli5

     |████████████████████████████████| 112kB 3.0MB/s 


In [0]:
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

In [0]:
HOME = '/content/drive/My Drive/Colab Notebooks/matrix/dataworkshop_matrix'
% cd {HOME}

/content/drive/My Drive/Colab Notebooks/matrix/dataworkshop_matrix


In [0]:
ls data

men_shoes.csv


In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)

In [0]:
df['brand_cat'] = df['brand'].factorize()[0]

In [0]:
def run_model(feats, model=DecisionTreeRegressor(max_depth=5)):
    X = df[ feats ].values
    y = df['prices_amountmin'].values
    score = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
    return np.mean(score), np.std(score)

In [12]:
run_model(['brand_cat'])

(-58.38655694633361, 4.223555478221712)

In [13]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.47223572384038, 4.328288468270897)

In [14]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [0]:
from ast import literal_eval
from tqdm import tqdm_notebook

In [0]:
def parse_feature(x):
    output_dict = dict()
    if str(x) == 'nan':
        return output_dict
    
    features = literal_eval(x.replace('\\"', '"'))
    output_dict = {x['key'].lower().strip(): x['value'][0].lower().strip() for x in features}
    
    return output_dict

df['parser_feature'] = df['features'].map(parse_feature)

In [22]:
df['parser_feature'].head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [23]:
keys = set()
df['parser_feature'].map(lambda x: keys.update(x.keys()))
len(keys)

476

In [0]:
def get_name_key(key):
    return 'feat_' + key

In [25]:
for key in tqdm_notebook(keys):
    df[get_name_key(key)] = df['parser_feature'].map(lambda x: x[key] if key in x else np.nan)

In [26]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_toe type', 'feat_материал', 'feat_design', 'feat_wheel type',
       'feat_band material', 'feat_material composition', 'feat_style name',
       'feat_watch band material', 'feat_mirrored',
       'feat_resolution megapixels'],
      dtype='object', length=526)

In [32]:
df [ df['feat_athlete'].isnull() == False ].shape[0] / df.shape[0] * 100

0.0437636761487965

In [0]:
key_stats = {}
for key in keys:
    key_stats[key] = df[df[get_name_key(key)].isnull() == False].shape[0] / df.shape[0] * 100

In [40]:
{k: v for k, v in key_stats.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]

for key in keys:
    df[get_name_key(key) + '_cat'] = df[get_name_key(key)].factorize()[0]

In [0]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
feats = ['brand_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_style_cat', 'feat_sport_cat', 'feat_shape_cat', 'feat_fit_cat']
feats_cat = [x for x in df.columns if x.endswith('_cat')]
feats += feats_cat
feats = list(set(feats))

result = run_model(feats, model)

In [77]:
print(result)

X = df[feats].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X, y)

perm = PermutationImportance(m, random_state=0).fit(X, y)
eli5.show_weights(perm, feature_names=feats)


(-57.687437701872476, 4.179266109889606)


Weight,Feature
0.2437 ± 0.0154,brand_cat
0.1063 ± 0.0130,feat_material_cat
0.0124 ± 0.0010,feat_weight_cat
0.0116 ± 0.0006,feat_brand_cat
0.0099 ± 0.0009,feat_fabric content_cat
0.0093 ± 0.0030,feat_adjustable_cat
0.0056 ± 0.0006,feat_shoe category_cat
0.0051 ± 0.0026,feat_resizable_cat
0.0046 ± 0.0008,feat_color_cat
0.0043 ± 0.0003,feat_fabric material_cat


In [0]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
# feats = ['brand_cat', 'feat_brand_cat', 'feat_material_cat', 'feat_fabric content_cat', 'feat_shape_cat', 'feat_weight_cat', 'feat_shoe category_cat']
feats = ['brand_cat', 'feat_brand_cat', 'feat_material_cat', 'feat_shape_cat', 'feat_weight_cat']

result = run_model(feats, model)

In [95]:
print(result)

X = df[feats].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X, y)

perm = PermutationImportance(m, random_state=0).fit(X, y)
eli5.show_weights(perm, feature_names=feats)

(-56.722350006588975, 4.1782280308029796)


Weight,Feature
0.2722 ± 0.0127,brand_cat
0.1147 ± 0.0100,feat_material_cat
0.0264 ± 0.0020,feat_shape_cat
0.0235 ± 0.0017,feat_brand_cat
0.0132 ± 0.0007,feat_weight_cat
